# 聚合与排序 Grouping & Sorting

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 5) # 让打印结果只显示5行（如果你希望显示全部，就把5改成None，不过小心数据量大内存爆掉~）
# 依旧是先读取红酒数据集
wine_reviews = pd.read_csv('winemag-data-50k-v2.csv', index_col = 0)
wine_reviews

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49998,Italy,The beautiful thing about this wine is the ric...,Le Rive,90,45.0,Veneto,Soave Classico,NaN,NaN,NaN,Suavia 2008 Le Rive (Soave Classico),Garganega,Suavia
49999,US,This is a particularly fine vintage for the po...,Dr. Wolfe's Family Red,90,15.0,Washington,Washington,Washington Other,Paul Gregutt,@paulgwine,Thurston Wolfe 2009 Dr. Wolfe's Family Red Red...,Red Blend,Thurston Wolfe


## groupby()
在日常数据分析中，经常需要将数据根据某个（多个）字段划分为不同的群体（group）进行分析，如电商领域将全国的总销售额根据省份来划分，分析各省销售额的变化情况，社交领域将用户画像（性别、年龄）进行细分，研究用户的使用情况和偏好等。<p>
在pandas中，上述数据处理操作主要运用 **groupby()** 完成。我们先来看个实例操作：

In [2]:
wine_reviews.groupby('points').points.count()

points
80     127
81     253
      ... 
99      11
100      7
Name: points, Length: 21, dtype: int64

上面这步所达到的效果是：将分数升序排列，并统计出对应的个数<p>
通俗来讲，**groupby**的过程就是将原有的**Dataframe**按照**groupby**的字段（这里是points）,划分为若干个**子Dataframe**。以上面的操作为例，points 取值范围是80-100，于是**groupby**操作将整个数据集划分成了21份子数据集（可以理解为按照不同分数取值进行了归类处理）。之后我们通过对 points 字段进行计数(count)操作，便可以得到每个分数在整个数据集中的数量。

还记得我们上一节中使用过的 value_counts() 吗？它其实就是 groupby() 操作的一个 shortcut

In [3]:
wine_reviews.points.value_counts() # 可以看到效果与上面一样，只是没有按照分数排序

87     6937
88     6446
       ... 
99       11
100       7
Name: points, Length: 21, dtype: int64

我们可以使用上一节所提到的任意 summary functions 来一起处理数据。比如我们想找到每个分数段最便宜的红酒：

In [4]:
wine_reviews.groupby('points').price.min()

points
80       5.0
81       5.0
       ...  
99      75.0
100    200.0
Name: price, Length: 21, dtype: float64

被 **groupby** 划分出来的每个子数据集都可以使用我们上一节学到的 **apply()** 函数。比如我们提取数据集中每个酒厂（winery字段）的第一款酒的名字：

In [5]:
wine_reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 12272, dtype: object

你甚至可以 group by 更多的列，例如我们根据国家和省份来寻找评分最棒的酒：

In [6]:
wine_reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
...                               ...   
Uruguay   San Jose            Uruguay   
          Uruguay             Uruguay   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If you love massive Argentine reds with purity...   
          Other             This single-vineyard Malbec blend from vineyar...   
...                                                                       ...   
Uruguay   San Jose          Baked, sweet, heavy aromas turn earthy with ti...   
          Uruguay           Cherry and berry aromas are ripe, healthy and ...   

                                                                  designation  \
country   province                                                              
Argentina Mendoza Province  Finca Pedregal Single Vineyard Barrancas Maipú...   
          Other                                                  Chañar Punco   
...                                                                       ...   
Uruguay   San Jose                                   El Preciado Gran Reserva   
          Uruguay                                   Blend 002 Limited Edition   

                            points  price          province          region_1  \
country   province                                                              
Argentina Mendoza Province      95   74.0  Mendoza Province           Mendoza   
          Other                 94   68.0             Other  Calchaquí Valley   
...                            ...    ...               ...               ...   
Uruguay   San Jose              87   50.0          San Jose               NaN   
          Uruguay               91   22.0           Uruguay               NaN   

                           region_2        taster_name taster_twitter_handle  \
country   province                                                             
Argentina Mendoza Province      NaN  Michael Schachner           @wineschach   
          Other                 NaN  Michael Schachner           @wineschach   
...                             ...                ...                   ...   
Uruguay   San Jose              NaN  Michael Schachner           @wineschach   
          Uruguay               NaN  Michael Schachner           @wineschach   

                                                                        title  \
country   province                                                              
Argentina Mendoza Province  Pascual Toso 2014 Finca Pedregal Single Vineya...   
          Other             El Esteco 2013 Chañar Punco Red (Calchaquí Val...   
...                                                                       ...   
Uruguay   San Jose          Castillo Viejo 2005 El Preciado Gran Reserva R...   
          Uruguay           Narbona NV Blend 002 Limited Edition Tannat-Ca...   

                                              variety          winery  
country   province                                                     
Argentina Mendoza Province  Cabernet Sauvignon-Malbec    Pascual Toso  
          Other                             Red Blend       El Esteco  
...                                               ...             ...  
Uruguay   San Jose                          Red Blend  Castillo Viejo  
          Uruguay               Tannat-Cabernet Franc         Narbona  

[363 rows x 13 columns]

上面用到的**idxmax()**会返回所选列中最大值对应的索引（index）

## 另一个值得一提的 groupby() 方法是 agg()
**agg()** 允许你可以同时调用多个函数

In [7]:
wine_reviews.groupby(['country']).price.agg([len, min, max]) 

,len,min,max
country,,,
Argentina,1465.0,4.0,215.0
Armenia,1.0,14.0,14.0
...,...,...,...
Ukraine,5.0,6.0,10.0
Uruguay,50.0,10.0,120.0


len 相当于统计了每个国家对应多少条数据，min max则是该国家所产红酒的最低和最高价格

## Multi-indexes
**groupby()** 操作还可以产生多个index

In [8]:
countries_reviewed = wine_reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  1255
          Other              210
...                          ...
Uruguay   San Jose             3
          Uruguay              7

[363 rows x 1 columns]

我们可以通过python中的 **type()** 函数来看看这个index的属性：

In [9]:
mi = countries_reviewed.index
type(mi)

pandas.core.indexes.multi.MultiIndex

可以看到上面的属性显示为MultiIndex，更多深入使用方法可以参考文档 [MultiIndex / Advanced Selection](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)，本篇入门教程中不做赘述<p>
通常来讲我们对于Multi-index所使用的最多操作是将其还原回单一index，这里我们只需要对Dataframe使用 **reset_index()** 就好：

In [10]:
countries_reviewed.reset_index()

,country,province,len
0,Argentina,Mendoza Province,1255
1,Argentina,Other,210
...,...,...,...
361,Uruguay,San Jose,3
362,Uruguay,Uruguay,7


## 排序（Sorting）
回顾刚才我们得到的countries_reviewed，可以看出聚合（grouping）操作返回的结果是按照索引顺序（index order）排序的。如果我们希望数据可以根据取值顺序进行排序，我们就需要手动去改变它。比如使用 **sort_values()**，你可以指定按照某一列的值来排序：

In [11]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
95,Croatia,North Dalmatia,1
97,Croatia,Podunavlje,1
...,...,...,...
353,US,Washington,3317
332,US,California,13741


与python中的 **sort()** 函数相同，**sort_values()** 函数也是默认按照升序（ascending order）排序。如果你想得到降序（descending order）结果，可以将 *ascending* 参数由默认的 True 改为 False ：

In [12]:
countries_reviewed.sort_values(by='len', ascending=False)

,country,province,len
332,US,California,13741
353,US,Washington,3317
...,...,...,...
88,Croatia,Croatia,1
268,Serbia,Fruška Gora,1


相对应的，你也可以使用 **sort_index()** 函数来按照索引排序，其参数设置与上面的 **sort_values()** 一样

In [13]:
countries_reviewed.sort_index()

,country,province,len
0,Argentina,Mendoza Province,1255
1,Argentina,Other,210
...,...,...,...
361,Uruguay,San Jose,3
362,Uruguay,Uruguay,7


#### 举一反三的时候又来了！你同样可以根据不止一列的值来排序：

In [14]:
countries_reviewed.sort_values(by=['country', 'len'])

,country,province,len
1,Argentina,Other,210
0,Argentina,Mendoza Province,1255
...,...,...,...
359,Uruguay,Montevideo,9
357,Uruguay,Canelones,18


其排序依据你输入的顺序来决定，先按照国家名称的字母序排序，之后按照len的值排序。

*好好消化，我们下一节再见~*